<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5_test_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Dec 19 10:52:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    48W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 28.2 MB/s 
     |████████████████████████████████| 1.9 MB 92.0 MB/s 
     |████████████████████████████████| 1.3 MB 100.2 MB/s 
     |████████████████████████████████| 9.2 MB 80.0 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 7.6 MB 78.1 MB/s 
     |████████████████████████████████| 5.8 MB 87.3 MB/s 
     |████████████████████████████████| 451 kB 85.6 MB/s 
     |████████████████████████████████| 182 kB 90.6 MB/s 
     |████████████████████████████████| 182 kB 97.5 MB/s 
     |████████████████████████████████| 174 kB 72.8 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 173 kB 81.8 MB/s 
     |████████████████████████████████| 168 kB 86.1 MB/s 
     |████████████████████████████████| 168 kB 81.5 MB/s 
     |████████████████████████████

In [4]:
import glob
 
dir_kpen = '/content/gdrive/MyDrive/DatasetKP/EN'
dir_kpen_list = []
dir_kpth = '/content/gdrive/MyDrive/DatasetKP/TH'
dir_kpth_list = []

for filename in glob.iglob(f'{dir_kpen}/*'):
    dir_kpen_list.append(filename)
for filename in glob.iglob(f'{dir_kpth}/*'):
    dir_kpth_list.append(filename)
dir_kpen_list_count = len(dir_kpen_list)
dir_kpth_list_count = len(dir_kpth_list)
dir_kpen_list.sort()
dir_kpth_list.sort()
print('datasetKP', dir_kpen_list_count, dir_kpth_list_count)

dir_myen = '/content/gdrive/MyDrive/DatasetMY/EN'
dir_myen_list = []
dir_myth = '/content/gdrive/MyDrive/DatasetMY/TH'
dir_myth_list = []

for filename in glob.iglob(f'{dir_myen}/*'):
    dir_myen_list.append(filename)
for filename in glob.iglob(f'{dir_myth}/*'):
    dir_myth_list.append(filename)
dir_myen_list_count = len(dir_myen_list)
dir_myth_list_count = len(dir_myth_list)
dir_myen_list.sort()
dir_myth_list.sort()
print('datasetMY', dir_myen_list_count, dir_myth_list_count)

if dir_kpen_list_count == dir_kpth_list_count:
  datasetKP = dir_kpen_list_count
if dir_myen_list_count == dir_myth_list_count:
  datasetMY = dir_myen_list_count
print(datasetKP, datasetMY)

datasetKP 157 157
datasetMY 80 80
157 80


In [5]:
import csv

with open('/content/dataset.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['prefix', 'input_text', 'target_text'])
  for i in range(datasetKP):
    with open(dir_kpen_list[i], 'r') as kpen:
      data_en = kpen.read()
    with open(dir_kpth_list[i], 'r') as kpth:
      data_th = kpth.read()
    writer.writerow(['translate thai to english', data_th, data_en])
  for i in range(datasetMY):
    with open(dir_myen_list[i], 'r') as myen:
      data_en = myen.read()
    with open(dir_myth_list[i], 'r') as myth:
      data_th = myth.read()
    writer.writerow(['translate thai to english', data_th, data_en])

import pandas as pd

pd.read_csv('/content/dataset.csv')

,prefix,input_text,target_text
0,translate thai to english,การเลือกที่ตั้งอุตสาหกรรมบนพื้นฐานของกฎเกณฑ์ทา...,Industrial site selection based on environmen...
1,translate thai to english,โรคเบต้า-ธาลัสซีเมียเป็นโรคที่เกิดจากการสร้างเ...,Augmentation of fetal hemoglobin (HbF) synthe...
2,translate thai to english,การวิจัยนี้เป็นการศึกษาเปรียบเทียบในกลุ่มศึกษา...,The purpose of this case-control study was wit...
3,translate thai to english,การศึกษาครั้งนี้มีวัตถุประสงค์เพื่อศึกษารูปแบบ...,This purpose of this study is to study the sel...
4,translate thai to english,การศึกษาครั้งนี้มีวัตถุประสงค์เพื่อศึกษา ผลการ...,The aims of the study were to investigate the ...
...,...,...,...
232,translate thai to english,บริษัท King of Valve จำกัด ดำเนินธุรกิจนำเข้าส...,This paper deals with determining an appropria...
233,translate thai to english,การเพิ่มประสิทธิภาพการทำงานเครื่องทอผ้าโดยการบ...,The study of working improvement of the weavin...
234,translate thai to english,การศึกษาครั้งนี้มีวัตถุประสงค์เพื่อสร้างรูปแบบ...,This study has object for making stainless she...
235,translate thai to english,สารนิพนธ์ฉบับนี้เป็นการศึกษาการพยากรณ์แบบอนุกร...,This paper deals with a study of forecasting t...


In [6]:
!rm -rf '/content/train.csv'
!rm -rf '/content/eval.csv'
!rm -rf '/content/test.csv'

from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/dataset.csv')
train_data, eval_data = train_test_split(data, train_size=200, shuffle=True)
train_data.to_csv('/content/train.csv', index=False)
eval_data[:30].to_csv('/content/eval.csv', index=False)
test_data = eval_data[30:]
test_data.to_csv('/content/test.csv', index=False)

In [7]:
pd.read_csv('/content/train.csv')

,prefix,input_text,target_text
0,translate thai to english,การวิจัยครั้งนี้มีวัตถุประสงค์เพื่อศึกษาระดับค...,The present study aimed to evaluate the level...
1,translate thai to english,การศึกษาครั้งนี้มีวัตถุประสงค์เพื่อสร้างรูปแบบ...,This study has object for making stainless she...
2,translate thai to english,ได้ทำการสังเคราะห์ L1H และ L2H สำหรับเป็นเซนเซ...,"Novel colorimetric sensors, L1H and L2H, deriv..."
3,translate thai to english,การเติบโตอย่างรวดเร็วของระบบผลิตไฟฟ้าพลังงานแส...,The growth in the adoption of solar photovolta...
4,translate thai to english,พื้นที่อำเภอแก่งคอยอยู่ในพื้นที่ลุ่มน้ำปาสักตอ...,"Amphoe Kaeng Khoi, study area is located in th..."
...,...,...,...
195,translate thai to english,การศึกษานี้ มีวัตถุประสงค์ เพื่อศึกษาอุดมการณ์...,The purpose of this research is to examine an ...
196,translate thai to english,การใช้พลังงานไฟฟ้ าในประเทศไทย ท้ังในภาคอุตสาห...,Electric energy in Thailand. In the industria...
197,translate thai to english,การจัดการศึกษาระดับอุดมศึกษาของมหาวิทยาลัยเทคโ...,Undergraduate educational management of Rajama...
198,translate thai to english,ผลจากการทดลองส่วนที่ 1 ศึกษาผลของตะกอนต่อคุณภา...,This research aims to develop the solid separa...


In [8]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/train.csv').astype(str)
eval_df = pd.read_csv('/content/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 128
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.num_train_epochs = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [9]:
# !wandb login --relogin

In [10]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

...

Running Epoch 6 of 8:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

(104,
 {'global_step': [13, 26, 39, 52, 65, 78, 91, 104],
  'eval_loss': [5.824857473373413,
   4.38957953453064,
   3.9805723428726196,
   3.8773711919784546,
   3.6802103519439697,
   3.5708224773406982,
   3.5603896379470825,
   3.585271954536438],
  'train_loss': [7.571951866149902,
   5.473212242126465,
   4.646483898162842,
   3.9907383918762207,
   3.825697422027588,
   3.7366368770599365,
   3.541485071182251,
   3.0632681846618652]})

In [11]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/30 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 256
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [13]:
valid = pd.read_csv("/content/eval.csv").astype(str)

# thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
# to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [14]:
english_preds = model.predict(to_english)

then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

Generating outputs:   0%|          | 0/4 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/30 [00:00<?, ?it/s]

Thai to English:  1.304391920004141


In [15]:
# from simpletransformers.t5 import T5Model

# # model = T5Model("mt5", "/content/outputs")
# to_predict = []
# data = pd.read_csv('/content/test.csv')
# for i, info in enumerate(data['input_text']):
#   to_predict.append(info)
# predictions = model.predict(to_predict)

In [16]:
# test_data['input_text']

In [17]:
# with open('/content/predict.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['ref_th', 'ref_eng', 'predict'])
#   for i in range(len(test_data)):
#     writer.writerow([test_data['input_text'][i], test_data['target_text'][i], predictions[i]])
# pd.read_csv('/content/predict.csv')